In [3]:
# %pip install -r requirements.txt

In [4]:
# Importing the necessary libraries
import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score
from ModelArchitecture.DiceLoss import dice_metric_loss
from ModelArchitecture import DUCK_Net
from ImageLoader import ImageLoader2D

In [5]:
# Checking the number of GPUs available

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-06-17 18:56:59.184884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:56:59.259673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:56:59.260438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
# Setting the model parameters

img_size = 352
dataset_type = 'chest' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 17 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

ct = datetime.now()

model_type = "DuckNet"

progress_path = 'ProgressFull/' + dataset_type + '_progress_csv_' + model_type + '_filters_' + str(filters) +  '_' + str(ct) + '.csv'
progressfull_path = 'ProgressFull/' + dataset_type + '_progress_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.txt'
plot_path = 'ProgressFull/' + dataset_type + '_progress_plot_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.png'
model_path = 'ModelSaveTensorFlow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)

EPOCHS = 600
min_loss_for_saving = 0.2

In [7]:
# Loading the data
X, Y = ImageLoader2D.load_data(img_size, img_size, -1, '')

/home/azimi2kht/projects/chest/RSUA Chest X-Ray Dataset/Data Chest X-Ray RSUA (Validated)/Covid/images/
Resizing training images and masks: 207


7it [00:00, 66.27it/s]

207it [00:02, 74.28it/s]


In [8]:
# Splitting the data, seed for reproducibility
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle= True, random_state = seed_value)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.111, shuffle= True, random_state = seed_value)

In [9]:
# Defining the augmentations

aug_train = albu.Compose([
    albu.HorizontalFlip(),
    albu.VerticalFlip(),
    albu.ColorJitter(brightness=(0.6,1.6), contrast=0.2, saturation=0.1, hue=0.01, always_apply=True),
    albu.Affine(scale=(0.5,1.5), translate_percent=(-0.125,0.125), rotate=(-180,180), shear=(-22.5,22), always_apply=True),
])

def augment_images():
    x_train_out = []
    y_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i])
        x_train_out.append(ug['image'])
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)

/home/azimi2kht/miniconda3/envs/fracatlas-model/lib/python3.9/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [10]:
# Creating the model

model = DUCK_Net.create_model(img_height=img_size, img_width=img_size, input_chanels=1, out_classes=1, starting_filters=filters)

2024-06-17 18:57:02.513053: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Starting DUCK-Net


2024-06-17 18:57:02.522847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:57:02.524762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:57:02.526143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:57:03.950662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-17 18:57:03.951565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [11]:
# Compiling the model

model.compile(optimizer=optimizer, loss=dice_metric_loss)

In [12]:
# Training the model

step = 0

for epoch in range(0, EPOCHS):

    print(f"Training, epoch {epoch}")
    print("Learning Rate: " + str(learning_rate))

    step += 1

    image_augmented, mask_augmented = augment_images()

    csv_logger = CSVLogger(progress_path, append=True, separator=";")

    model.fit(
        x=image_augmented,
        y=mask_augmented,
        epochs=1,
        batch_size=4,
        validation_data=(x_valid, y_valid),
        verbose=1,
        callbacks=[csv_logger],
    )

    prediction_valid = model.predict(x_valid, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid)

    loss_valid = loss_valid.numpy()
    print("Loss Validation: " + str(loss_valid))

    prediction_test = model.predict(x_test, verbose=0)
    loss_test = dice_metric_loss(y_test, prediction_test)
    loss_test = loss_test.numpy()
    print("Loss Test: " + str(loss_test))

    with open(progressfull_path, "a") as f:
        f.write(
            "epoch: "
            + str(epoch)
            + "\nval_loss: "
            + str(loss_valid)
            + "\ntest_loss: "
            + str(loss_test)
            + "\n\n\n"
        )

    if min_loss_for_saving > loss_valid:
        min_loss_for_saving = loss_valid
        print("Saved model with val_loss: ", loss_valid)
        model.save(model_path)

    del image_augmented
    del mask_augmented

    gc.collect()

Training, epoch 0
Learning Rate: 0.0001


2024-06-17 18:59:54.685284: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8907
2024-06-17 18:59:56.396172: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


42/42 [==============================] - 206s 1s/step - loss: 0.5813 - val_loss: 0.6851
Loss Validation: 0.68496037
Loss Test: 0.63917065
Training, epoch 1
Learning Rate: 0.0001
42/42 [==============================] - 35s 841ms/step - loss: 0.4633 - val_loss: 0.6823
Loss Validation: 0.68199503
Loss Test: 0.63053864
Training, epoch 2
Learning Rate: 0.0001
42/42 [==============================] - 35s 841ms/step - loss: 0.3922 - val_loss: 0.6414
Loss Validation: 0.641269
Loss Test: 0.5910156
Training, epoch 3
Learning Rate: 0.0001
42/42 [==============================] - 35s 838ms/step - loss: 0.3538 - val_loss: 0.6295
Loss Validation: 0.6299359
Loss Test: 0.5822644
Training, epoch 4
Learning Rate: 0.0001
42/42 [==============================] - 35s 837ms/step - loss: 0.3262 - val_loss: 0.5769
Loss Validation: 0.5774499
Loss Test: 0.5270659
Training, epoch 5
Learning Rate: 0.0001
42/42 [==============================] - 35s 841ms/step - loss: 0.3088 - val_loss: 0.5269
Loss Validation: 0.

INFO:tensorflow:Assets written to: ModelSaveTensorFlow/chest/DuckNet_filters_17_2024-06-17 18:56:59.302768/assets


INFO:tensorflow:Assets written to: ModelSaveTensorFlow/chest/DuckNet_filters_17_2024-06-17 18:56:59.302768/assets


...

INFO:tensorflow:Assets written to: ModelSaveTensorFlow/chest/DuckNet_filters_17_2024-06-17 18:56:59.302768/assets


Training, epoch 363
Learning Rate: 0.0001
42/42 [==============================] - 32s 766ms/step - loss: 0.0332 - val_loss: 0.0358
Loss Validation: 0.035611153
Loss Test: 0.032792866
Training, epoch 364
Learning Rate: 0.0001
42/42 [==============================] - 32s 770ms/step - loss: 0.0346 - val_loss: 0.0291
Loss Validation: 0.028931618
Loss Test: 0.0322724
Training, epoch 365
Learning Rate: 0.0001
42/42 [==============================] - 32s 774ms/step - loss: 0.0321 - val_loss: 0.0289
Loss Validation: 0.028797626
Loss Test: 0.030531406
Training, epoch 366
Learning Rate: 0.0001
42/42 [==============================] - 32s 772ms/step - loss: 0.0329 - val_loss: 0.0316
Loss Validation: 0.03140092
Loss Test: 0.029101431
Training, epoch 367
Learning Rate: 0.0001
42/42 [==============================] - 32s 773ms/step - loss: 0.0337 - val_loss: 0.0304
Loss Validation: 0.030334175
Loss Test: 0.030275166
Training, epoch 368
Learning Rate: 0.0001
42/42 [==============================] - 

INFO:tensorflow:Assets written to: ModelSaveTensorFlow/chest/DuckNet_filters_17_2024-06-17 18:56:59.302768/assets


INFO:tensorflow:Assets written to: ModelSaveTensorFlow/chest/DuckNet_filters_17_2024-06-17 18:56:59.302768/assets


Training, epoch 418
Learning Rate: 0.0001
42/42 [==============================] - 32s 769ms/step - loss: 0.0325 - val_loss: 0.0287
Loss Validation: 0.028871536
Loss Test: 0.028085232
Training, epoch 419
Learning Rate: 0.0001
42/42 [==============================] - 33s 777ms/step - loss: 0.0317 - val_loss: 0.0290
Loss Validation: 0.028868318
Loss Test: 0.028774202
Training, epoch 420
Learning Rate: 0.0001
42/42 [==============================] - 32s 772ms/step - loss: 0.0323 - val_loss: 0.0269
Loss Validation: 0.02695942
Loss Test: 0.03046
Training, epoch 421
Learning Rate: 0.0001
42/42 [==============================] - 32s 772ms/step - loss: 0.0335 - val_loss: 0.0262
Loss Validation: 0.02604711
Loss Test: 0.028243184
Training, epoch 422
Learning Rate: 0.0001
42/42 [==============================] - 32s 774ms/step - loss: 0.0309 - val_loss: 0.0286
Loss Validation: 0.028586924
Loss Test: 0.028958976
Training, epoch 423
Learning Rate: 0.0001
42/42 [==============================] - 33s

In [14]:
# Computing the metrics and saving the results

print("Loading the model")

model = tf.keras.models.load_model(model_path, custom_objects={'dice_metric_loss':dice_metric_loss})

prediction_train = model.predict(x_train, batch_size=4)
prediction_valid = model.predict(x_valid, batch_size=4)
prediction_test = model.predict(x_test, batch_size=4)

print("Predictions done")

dice_train = f1_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
dice_test = f1_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
dice_valid = f1_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))
print(dice_train)
print(dice_valid)
print(dice_test)

print("Dice finished")


miou_train = jaccard_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
miou_test = jaccard_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
miou_valid = jaccard_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print(miou_train)
print(miou_valid)
print(miou_test)

print("Miou finished")


precision_train = precision_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                  np.ndarray.flatten(prediction_train > 0.5))
precision_test = precision_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
precision_valid = precision_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                  np.ndarray.flatten(prediction_valid > 0.5))
print(precision_train)
print(precision_valid)
print(precision_test)
print("Precision finished")


recall_train = recall_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                            np.ndarray.flatten(prediction_train > 0.5))
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))
print(recall_train)
print(recall_valid)
print(recall_test)
print("Recall finished")


accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))


print(accuracy_train)
print(accuracy_valid)
print(accuracy_test)
print("Accuracy finished")


final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('dice_train: ' + str(dice_train) + ' dice_valid: ' + str(dice_valid) + ' dice_test: ' + str(dice_test) + '\n\n')
    f.write('miou_train: ' + str(miou_train) + ' miou_valid: ' + str(miou_valid) + ' miou_test: ' + str(miou_test) + '\n\n')
    f.write('precision_train: ' + str(precision_train) + ' precision_valid: ' + str(precision_valid) + ' precision_test: ' + str(precision_test) + '\n\n')
    f.write('recall_train: ' + str(recall_train) + ' recall_valid: ' + str(recall_valid) + ' recall_test: ' + str(recall_test) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_valid: ' + str(accuracy_valid) + ' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')

Loading the model
6/6 [==============================] - 1s 190ms/step
Predictions done
0.9764794087799983
0.9749626692225737
0.9737673495263708
Dice finished
0.9540398279785147
0.9511484508404449
0.9488758217514863
Miou finished
0.9839473579573375
0.9755706526772465
0.9781354923037355
Precision finished
0.9691239659650561
0.9743554430962575
0.9694380476665194
Recall finished
0.990728599267468
0.9906432937327824
0.9885022352174341
Accuracy finished
results_DuckNet_17_chest.txt
File done
